## Transposed Convolution
Transposed convolutions also called fractionally strided convolutions or deconvolutions work by swapping the forward and backward passes of a convolution. One way to put it is to note that the kernel defines a convolution, but whether it’s a direct convolution or a transposed convolution is determined by how the forward and backward passes are computed [1]. 

For instance, although the kernel $\mathbf{w}$ defines a convolution whose forward and backward passes are computed by multiplying with $\mathbf{C}$ and $\mathbf{C}^T$ respectively, it also defines a transposed convolution whose forward and backward passes are computed by multiplying with $\mathbf{C}^T$ and $(\mathbf{C}^T)^T = \mathbf{C}$ respectively.

It is defined as SpatialFullConvolution in Torch

[1]: Vincent Dumoulin, Francesco Visin - [A guide to convolution arithmetic for deep learning](https://arxiv.org/abs/1603.07285 "A guide to convolution arithmetic for deep learning") 

### SpatialFullConvolution
`module = nn.SpatialFullConvolution(nInputPlane, nOutputPlane, kW, kH, [dW], [dH], [padW], [padH], [adjW], [adjH])`

Other frameworks call this operation "In-network Upsampling", "Fractionally-strided convolution", "Backwards Convolution," "Deconvolution", or "Upconvolution."
The parameters are the following:
* `nInputPlane:` The number of expected input planes in the image given into `forward()`.
* `nOutputPlane:` The number of output planes the convolution layer will produce.
* `kW:` The kernel width of the convolution
* `kH:` The kernel height of the convolution
* `dW:` The step of the convolution in the width dimension. Default is 1.
* `dH:` The step of the convolution in the height dimension. Default is 1.
* `padW:` Additional zeros added to the input plane data on both sides of width axis. Default is 0. (kW-1)/2 is often used here.
* `padH:` Additional zeros added to the input plane data on both sides of height axis. Default is 0. (kH-1)/2 is often used here.
* `adjW:` Extra width to add to the output image. Default is 0. Cannot be greater than dW-1.
* `adjH:` Extra height to add to the output image. Default is 0. Cannot be greater than dH-1.


In [1]:
require 'torch'
require 'nn'

### (Normal) Convolution Forward & Backward Pass

Layer with kernel size $3 \times 3$ and stride of $2$, $2$

In [2]:
conv = nn.SpatialConvolutionMM(1,1,3,3,2,2)
conv.weight:uniform(0,2)
conv.bias:fill(0)
print(conv.weight)

 1.0160  0.4442  0.8234  1.7105  1.9389  1.8970  0.4721  1.4563  0.4023
[torch.DoubleTensor of size 1x9]



#### Input
Image of channel $1$ and size $5 \times 5$, $r$ is the inputGradient

In [3]:
imgC = torch.Tensor(1,1,5,5)
imgC:uniform(0,5)

In [4]:
r=conv:forward(imgC)
print(r)

(1,1,.,.) = 
  28.0467  21.1382
  23.1202  30.5335
[torch.DoubleTensor of size 1x1x2x2]



In [5]:
conv:backward(imgC,r)

(1,1,.,.) = 
  28.4956  12.4596  44.5710   9.3905  17.4058
  47.9750  54.3784  89.3627  40.9838  40.0994
  36.7311  51.1151  71.3240  44.3476  33.6471
  39.5481  44.8267  96.0881  59.1999  57.9224
  10.9150  33.6697  23.7172  44.4655  12.2851
[torch.DoubleTensor of size 1x1x5x5]



### Transpose Convolution Forward Pass

Layer with kernel size $3 \times 3$ and stride of $2$, $2$

In [6]:
trancon = nn.SpatialFullConvolution(1, 1, 3, 3, 2, 2)
trancon.bias:fill(0)

In [7]:
trancon.weight:copy(conv.weight)
trancon.weight:reshape(1,1,3,3)
print(trancon.weight)

(1,1,.,.) = 
  1.0160  0.4442  0.8234
  1.7105  1.9389  1.8970
  0.4721  1.4563  0.4023
[torch.DoubleTensor of size 1x1x3x3]



#### Input
Image of channel $1$ and size $2 \times 2$ 

In [8]:
print(r)

(1,1,.,.) = 
  28.0467  21.1382
  23.1202  30.5335
[torch.DoubleTensor of size 1x1x2x2]



#### Forward Pass - Note this is exactly same as the backward pass of (normal) convolution!

In [9]:
trancon:forward(r)

(1,1,.,.) = 
  28.4956  12.4596  44.5710   9.3905  17.4058
  47.9750  54.3784  89.3627  40.9838  40.0994
  36.7311  51.1151  71.3240  44.3476  33.6471
  39.5481  44.8267  96.0881  59.1999  57.9224
  10.9150  33.6697  23.7172  44.4655  12.2851
[torch.DoubleTensor of size 1x1x5x5]

